### Project Proposal

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By # used to import different ways to access data in the XML or HTML file
from selenium.webdriver.chrome.service import Service # no longer need to download a driver file, use service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager # used to manage the Chrome driver to emulate a Chrome web browser

import time
import random

In [3]:
# Load the Teams.csv file 
teams_df = pd.read_csv('team.csv')

# Display the first few rows
print(teams_df.head())

   year league_id team_id franchise_id div_id  rank   g  ghome   w   l  ...  \
0  1871       NaN     BS1          BNA    NaN     3  31    NaN  20  10  ...   
1  1871       NaN     CH1          CNA    NaN     2  28    NaN  19   9  ...   
2  1871       NaN     CL1          CFC    NaN     8  29    NaN  10  19  ...   
3  1871       NaN     FW1          KEK    NaN     7  19    NaN   7  12  ...   
4  1871       NaN     NY2          NNA    NaN     5  33    NaN  16  17  ...   

   dp    fp                     name                          park  \
0 NaN  0.83     Boston Red Stockings           South End Grounds I   
1 NaN  0.82  Chicago White Stockings       Union Base-Ball Grounds   
2 NaN  0.81   Cleveland Forest Citys  National Association Grounds   
3 NaN  0.80     Fort Wayne Kekiongas                Hamilton Field   
4 NaN  0.83         New York Mutuals      Union Grounds (Brooklyn)   

   attendance  bpf  ppf  team_id_br  team_id_lahman45  team_id_retro  
0         NaN  103   98         B

In [5]:
# List all available columns
print(teams_df.columns.tolist())

['year', 'league_id', 'team_id', 'franchise_id', 'div_id', 'rank', 'g', 'ghome', 'w', 'l', 'div_win', 'wc_win', 'lg_win', 'ws_win', 'r', 'ab', 'h', 'double', 'triple', 'hr', 'bb', 'so', 'sb', 'cs', 'hbp', 'sf', 'ra', 'er', 'era', 'cg', 'sho', 'sv', 'ipouts', 'ha', 'hra', 'bba', 'soa', 'e', 'dp', 'fp', 'name', 'park', 'attendance', 'bpf', 'ppf', 'team_id_br', 'team_id_lahman45', 'team_id_retro']


In [7]:
# Select relevant columns using the actual column names in your file
teams_cleaned = teams_df[[
    'year', 'team_id', 'div_id', 'park', 'rank', 'w', 'l', 'g',
    'hr', 'bb', 'so', 'r', 'h', 'ab', 'div_win', 'wc_win', 'lg_win', 'ws_win',
]]

# Optional: Rename columns for clarity
teams_cleaned = teams_cleaned.rename(columns={
    'year': 'Year',
    'team_id': 'TeamID',
    'div_id': 'Division',
    'park' : 'BallPark',
    'rank' : 'DivisionRank',
    'w': 'Wins',
    'l': 'Losses',
    'g': 'Games',
    'hr': 'HomeRuns',
    'bb' : 'Walks',
    'so' : 'Strikeouts',
    'r': 'Runs',
    'h': 'Hits',
    'ab': 'AtBats',
    'div_win' : 'DivisionWinner',
    'wc_win' : 'WildCardWinner',
    'lg_win' : 'LeagueWinner',
    'ws_win' : 'WorldSeriesWinner'
})

# Preview the cleaned DataFrame
print(teams_cleaned.tail(5))


      Year TeamID Division        BallPark  DivisionRank  Wins  Losses  Games  \
2800  2015    LAN        W  Dodger Stadium             1    92      70    162   
2801  2015    SFN        W       AT&T Park             2    84      78    162   
2802  2015    ARI        W     Chase Field             3    79      83    162   
2803  2015    SDN        W      Petco Park             4    74      88    162   
2804  2015    COL        W     Coors Field             5    68      94    162   

      HomeRuns  Walks  Strikeouts  Runs  Hits  AtBats DivisionWinner  \
2800       187    563      1258.0   667  1346    5385              Y   
2801       136    457      1159.0   696  1486    5565              N   
2802       154    490      1312.0   720  1494    5649              N   
2803       148    426      1327.0   650  1324    5457              N   
2804       186    388      1283.0   737  1479    5572              N   

     WildCardWinner LeagueWinner WorldSeriesWinner  
2800              N        

In [9]:
df_1990_2015 = teams_cleaned[teams_cleaned['Year'].between(1990, 2015)].reset_index(drop=True)

df_1990_2015['DivisionRank'] = df_1990_2015['DivisionRank'].astype(int)
df_1990_2015['Wins'] = df_1990_2015['Wins'].astype(int)
df_1990_2015['Losses'] = df_1990_2015['Losses'].astype(int)
print(df_1990_2015.head())

   Year TeamID Division                       BallPark  DivisionRank  Wins  \
0  1990    ATL        W  Atlanta-Fulton County Stadium             6    65   
1  1990    BAL        E               Memorial Stadium             5    76   
2  1990    BOS        E                 Fenway Park II             1    88   
3  1990    CAL        W                Anaheim Stadium             4    80   
4  1990    CHA        W                  Comiskey Park             2    94   

   Losses  Games  HomeRuns  Walks  Strikeouts  Runs  Hits  AtBats  \
0      97    162       162    473      1010.0   682  1376    5504   
1      85    161       132    660       962.0   669  1328    5410   
2      74    162       106    598       795.0   699  1502    5516   
3      82    162       147    566      1000.0   690  1448    5570   
4      68    162       106    478       903.0   682  1393    5402   

  DivisionWinner WildCardWinner LeagueWinner WorldSeriesWinner  
0              N            NaN            N       

### Scraping 2016 batting stats as a test to see if it works

In [12]:
#scraping 2016 as a test
import pandas as pd

# Load the 2016 team batting table
url = "https://www.baseball-reference.com/leagues/majors/2016-standard-batting.shtml"
tables = pd.read_html(url)

# Grab the batting table that contains the 'Tm' column
batting_df = None
for table in tables:
    if 'Tm' in table.columns:
        batting_df = table
        break

# Drop League Total row
batting_df = batting_df[batting_df['Tm'] != 'League Total']

# Add Year column
batting_df['Year'] = 2016

# Rename columns to match your structure where possible
batting_df = batting_df.rename(columns={
    'Tm': 'TeamName',
    'R': 'Runs',
    'HR': 'HomeRuns',
    'BB': 'Walks',
    'SO': 'Strikeouts',
    'H': 'Hits',
    'AB': 'AtBats',
    'G': 'Games'
})

# Select the matching columns that actually exist
selected_columns = [
    'Year',         
    'TeamName',     
    'Games', 
    'HomeRuns',
    'Walks',
    'Strikeouts',
    'Runs',
    'Hits',
    'AtBats'
]

batting_df = batting_df[selected_columns]

# Preview
print(batting_df.head())





   Year              TeamName Games HomeRuns Walks Strikeouts Runs  Hits  \
0  2016  Arizona Diamondbacks   162      190   463       1427  752  1479   
1  2016        Atlanta Braves   161      122   502       1240  649  1404   
2  2016     Baltimore Orioles   162      253   468       1324  744  1413   
3  2016        Boston Red Sox   162      208   558       1160  878  1598   
4  2016          Chicago Cubs   162      199   656       1339  808  1409   

  AtBats  
0   5665  
1   5514  
2   5524  
3   5670  
4   5503  


### Scraping 2016-2024

In [15]:
from io import StringIO
import time
import random

all_years = []

for year in range(2016, 2025):
    try:
        url = f"https://www.baseball-reference.com/leagues/majors/{year}-standard-batting.shtml"
        tables = pd.read_html(url)
        
        # Find table with 'Tm' column
        batting_df = None
        for table in tables:
            if 'Tm' in table.columns:
                batting_df = table
                break

        if batting_df is None:
            raise ValueError("Table with 'Tm' column not found.")

        # Drop 'League Total' row
        batting_df = batting_df[batting_df['Tm'] != 'League Total']

        # Add Year column
        batting_df['Year'] = year

        # Rename to match format
        batting_df = batting_df.rename(columns={
            'Tm': 'TeamName',
            'R': 'Runs',
            'HR': 'HomeRuns',
            'BB': 'Walks',
            'SO': 'Strikeouts',
            'H': 'Hits',
            'AB': 'AtBats',
            'G': 'Games'
        })

        # Select only the desired columns
        selected_columns = [
            'Year', 'TeamName', 'Games', 'HomeRuns', 'Walks',
            'Strikeouts', 'Runs', 'Hits', 'AtBats'
        ]

        batting_df = batting_df[selected_columns]
        all_years.append(batting_df)

        print(f"Scraped {year}")
        time.sleep(random.uniform(2, 4))

    except Exception as e:
        print(f"Failed to scrape {year}: {e}")


Scraped 2016
Scraped 2017
Scraped 2018
Scraped 2019
Scraped 2020
Scraped 2021
Scraped 2022
Scraped 2023
Scraped 2024


In [16]:
# Combine all years into one DataFrame
df_2016_2024 = pd.concat(all_years).reset_index(drop=True)

# Show the full range of years
print("Years included:", df_2016_2024['Year'].unique())

# Preview the first and last few rows
print("\n First 5 rows:")
print(df_2016_2024.head())



Years included: [2016 2017 2018 2019 2020 2021 2022 2023 2024]

 First 5 rows:
   Year              TeamName Games HomeRuns Walks Strikeouts Runs  Hits  \
0  2016  Arizona Diamondbacks   162      190   463       1427  752  1479   
1  2016        Atlanta Braves   161      122   502       1240  649  1404   
2  2016     Baltimore Orioles   162      253   468       1324  744  1413   
3  2016        Boston Red Sox   162      208   558       1160  878  1598   
4  2016          Chicago Cubs   162      199   656       1339  808  1409   

  AtBats  
0   5665  
1   5514  
2   5524  
3   5670  
4   5503  


### Add TeamID to the Scraped Data 

In [20]:
df_2016_2024 = df_2016_2024[
    ~df_2016_2024['TeamName'].isin(['League Average', 'Tm'])
]
df_2016_2024 = df_2016_2024[df_2016_2024['TeamName'].notna()]

# Team name to Lahman-style team_id
team_name_to_id = {
    'Arizona Diamondbacks': 'ARI',
    'Atlanta Braves': 'ATL',
    'Baltimore Orioles': 'BAL',
    'Boston Red Sox': 'BOS',
    'Chicago White Sox': 'CHW',
    'Chicago Cubs': 'CHC',
    'Cincinnati Reds': 'CIN',
    'Cleveland Guardians': 'CLE',
    'Cleveland Indians': 'CLE',
    'Colorado Rockies': 'COL',
    'Detroit Tigers': 'DET',
    'Houston Astros': 'HOU',
    'Kansas City Royals': 'KCR',
    'Los Angeles Angels': 'LAA',
    'Los Angeles Dodgers': 'LAD',
    'Miami Marlins': 'MIA',
    'Milwaukee Brewers': 'MIL',
    'Minnesota Twins': 'MIN',
    'New York Mets': 'NYM',
    'New York Yankees': 'NYY',
    'Oakland Athletics': 'OAK',
    'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT',
    'San Diego Padres': 'SDP',
    'San Francisco Giants': 'SFG',
    'Seattle Mariners': 'SEA',
    'St. Louis Cardinals': 'STL',
    'Tampa Bay Rays': 'TBR',
    'Texas Rangers': 'TEX',
    'Toronto Blue Jays': 'TOR',
    'Washington Nationals': 'WSN'
}

# Apply the mapping
df_2016_2024['TeamID'] = df_2016_2024['TeamName'].map(team_name_to_id)

# Check for any unmapped teams
unmapped = df_2016_2024[df_2016_2024['TeamID'].isna()]['TeamName'].unique()
print("Unmapped teams:", unmapped)



Unmapped teams: []


### Reordering Columns to Match

In [23]:
df_2016_2024 = df_2016_2024[[
    'Year', 'TeamID', 'Games', 'HomeRuns', 'Walks',
    'Strikeouts', 'Runs', 'Hits', 'AtBats'
]]

print(df_2016_2024.head())

   Year TeamID Games HomeRuns Walks Strikeouts Runs  Hits AtBats
0  2016    ARI   162      190   463       1427  752  1479   5665
1  2016    ATL   161      122   502       1240  649  1404   5514
2  2016    BAL   162      253   468       1324  744  1413   5524
3  2016    BOS   162      208   558       1160  878  1598   5670
4  2016    CHC   162      199   656       1339  808  1409   5503


### Merging the dataframes

In [26]:
# Merge the two DataFrames
final_df = pd.concat([df_1990_2015, df_2016_2024], ignore_index=True)

# Preview the result
print("Final merged dataset shape:", final_df.shape)
print(final_df.head())
print(final_df.tail())

final_df.to_csv("MLB_Data1", encoding='utf-8',index=False)

Final merged dataset shape: (1028, 18)
   Year TeamID Division                       BallPark  DivisionRank  Wins  \
0  1990    ATL        W  Atlanta-Fulton County Stadium           6.0  65.0   
1  1990    BAL        E               Memorial Stadium           5.0  76.0   
2  1990    BOS        E                 Fenway Park II           1.0  88.0   
3  1990    CAL        W                Anaheim Stadium           4.0  80.0   
4  1990    CHA        W                  Comiskey Park           2.0  94.0   

   Losses Games HomeRuns Walks Strikeouts Runs  Hits AtBats DivisionWinner  \
0    97.0   162      162   473     1010.0  682  1376   5504              N   
1    85.0   161      132   660      962.0  669  1328   5410              N   
2    74.0   162      106   598      795.0  699  1502   5516              Y   
3    82.0   162      147   566     1000.0  690  1448   5570              N   
4    68.0   162      106   478      903.0  682  1393   5402              N   

  WildCardWinner League

In [28]:
team_info = {
    "ARI": {"Division": "W", "Ballpark": "Chase Field"},
    "ATL": {"Division": "E", "Ballpark": "Truist Park"},
    "BAL": {"Division": "E", "Ballpark": "Oriole Park at Camden Yards"},
    "BOS": {"Division": "E", "Ballpark": "Fenway Park"},
    "CHW": {"Division": "C", "Ballpark": "Guaranteed Rate Field"},
    "CHC": {"Division": "C", "Ballpark": "Wrigley Field"},
    "CIN": {"Division": "C", "Ballpark": "Great American Ball Park"},
    "CLE": {"Division": "C", "Ballpark": "Progressive Field"},
    "COL": {"Division": "W", "Ballpark": "Coors Field"},
    "DET": {"Division": "C", "Ballpark": "Comerica Park"},
    "HOU": {"Division": "W", "Ballpark": "Minute Maid Park"},
    "KCR": {"Division": "C", "Ballpark": "Kauffman Stadium"},
    "LAA": {"Division": "W", "Ballpark": "Angel Stadium"},
    "LAD": {"Division": "W", "Ballpark": "Dodger Stadium"},
    "MIA": {"Division": "E", "Ballpark": "loanDepot Park"},
    "MIL": {"Division": "C", "Ballpark": "American Family Field"},
    "MIN": {"Division": "C", "Ballpark": "Target Field"},
    "NYM": {"Division": "E", "Ballpark": "Citi Field"},
    "NYY": {"Division": "E", "Ballpark": "Yankee Stadium"},
    "OAK": {"Division": "W", "Ballpark": "Oakland Coliseum"},
    "PHI": {"Division": "E", "Ballpark": "Citizens Bank Park"},
    "PIT": {"Division": "C", "Ballpark": "PNC Park"},
    "SDP": {"Division": "W", "Ballpark": "Petco Park"},
    "SFG": {"Division": "W", "Ballpark": "Oracle Park"},
    "SEA": {"Division": "W", "Ballpark": "T-Mobile Park"},
    "STL": {"Division": "C", "Ballpark": "Busch Stadium"},
    "TBR": {"Division": "E", "Ballpark": "Tropicana Field"},
    "TEX": {"Division": "W", "Ballpark": "Globe Life Field"},
    "TOR": {"Division": "E", "Ballpark": "Rogers Centre"},
    "WSN": {"Division": "E", "Ballpark": "Nationals Park"},
}
# Add these to your team_info dictionary:

team_info.update({
    "CAL": {"Division": "W", "Ballpark": "Anaheim Stadium"},              # California Angels (pre-1997)
    "CHA": {"Division": "C", "Ballpark": "Comiskey Park / Guaranteed Rate Field"},  # Chicago White Sox (CHA = alt. Lahman ID for CHW)
    "CHN": {"Division": "C", "Ballpark": "Wrigley Field"},               # Chicago Cubs (CHN = alt. Lahman ID for CHC)
    "LAN": {"Division": "W", "Ballpark": "Dodger Stadium"},              # LA Dodgers (LAN = alt. Lahman ID for LAD)
    "KCA": {"Division": "C", "Ballpark": "Kauffman Stadium"},            # KC Royals (KCA = alt. Lahman ID for KCR)
    "SDN": {"Division": "W", "Ballpark": "Petco Park / Qualcomm Stadium"}, # San Diego Padres (SDN = alt. Lahman ID for SDP)
    "SFN": {"Division": "W", "Ballpark": "Candlestick Park / Oracle Park"},    # San Francisco Giants
    "SLN": {"Division": "C", "Ballpark": "Busch Stadium"},                     # St. Louis Cardinals
    "ML4": {"Division": "C", "Ballpark": "Milwaukee County Stadium"},         # Milwaukee Brewers (before Miller Park/AmFam Field)
    "MON": {"Division": "E", "Ballpark": "Olympic Stadium"},                  # Montreal Expos (moved to become WSN in 2005)
    "NYA": {"Division": "E", "Ballpark": "Yankee Stadium"},                   # New York Yankees
    "NYN": {"Division": "E", "Ballpark": "Shea Stadium / Citi Field"},
    "FLO": {"Division": "E", "Ballpark": "Pro Player Stadium / Sun Life Stadium"},
    "ANA": {"Division": "W", "Ballpark": "Angel Stadium"},         # Anaheim Angels (1997–2004)
    "TBA": {"Division": "E", "Ballpark": "Tropicana Field"},
    "WAS": {"Division": "E", "Ballpark": "Nationals Park"}  # Washington Nationals (2005–present)
})


# Overwrite existing 'Division' and 'BallPark' columns based on TeamID
final_df['Division'] = final_df['TeamID'].map(lambda x: team_info.get(x, {}).get('Division', 'Unknown'))
final_df['BallPark'] = final_df['TeamID'].map(lambda x: team_info.get(x, {}).get('Ballpark', 'Unknown'))

In [30]:
# Map of TeamID to League (Conference)
team_league = {
    "ARI": "NL", "ATL": "NL", "BAL": "AL", "BOS": "AL", "CHW": "AL", "CHC": "NL", "CIN": "NL",
    "CLE": "AL", "COL": "NL", "DET": "AL", "HOU": "AL", "KCR": "AL", "LAA": "AL", "LAD": "NL",
    "MIA": "NL", "MIL": "NL", "MIN": "AL", "NYM": "NL", "NYY": "AL", "OAK": "AL", "PHI": "NL",
    "PIT": "NL", "SDP": "NL", "SEA": "AL", "SFG": "NL", "STL": "NL", "TBR": "AL", "TEX": "AL",
    "TOR": "AL", "WSN": "NL", "CAL": "AL", "CHA": "AL", "CHN": "NL", "LAN": "NL", "KCA": "AL",
    "SDN": "NL", "SFN": "NL", "SLN": "NL", "ML4": "AL", "MON": "NL", "NYA": "AL", "NYN": "NL",
    "FLO": "NL", "ANA": "AL", "TBA": "AL", "WAS": "NL"
}

# Create a new column 'Conference' using the TeamID
final_df['Conference'] = final_df['TeamID'].map(lambda x: team_league.get(x, 'Unknown'))


In [32]:
# Move 'Conference' before 'Division'
cols = list(final_df.columns)
cols.insert(cols.index('Division'), cols.pop(cols.index('Conference')))
final_df = final_df[cols]

In [34]:
# Drop unnecessary columns
final_df = final_df.drop(columns=['DivisionRank', 'WildCardWinner', 'TeamName', 'Ballpark'])


KeyError: "['TeamName', 'Ballpark'] not found in axis"

In [36]:
# --- Final Cleaned Division Winners ---
division_winners = {
    (1990, 'BOS'): 'Y', (1990, 'OAK'): 'Y', (1990, 'PIT'): 'Y',
    (1991, 'MIN'): 'Y', (1991, 'TOR'): 'Y', (1991, 'ATL'): 'Y',
    (1992, 'OAK'): 'Y', (1992, 'TOR'): 'Y', (1992, 'ATL'): 'Y',
    (1993, 'CHW'): 'Y', (1993, 'PHI'): 'Y', (1993, 'ATL'): 'Y',
    (1995, 'BOS'): 'Y', (1995, 'CLE'): 'Y', (1995, 'SEA'): 'Y', (1995, 'ATL'): 'Y', (1995, 'CIN'): 'Y', (1995, 'LAD'): 'Y',
    (1996, 'NYY'): 'Y', (1996, 'CLE'): 'Y', (1996, 'TEX'): 'Y', (1996, 'ATL'): 'Y', (1996, 'STL'): 'Y', (1996, 'SDN'): 'Y',
    (1997, 'BAL'): 'Y', (1997, 'CLE'): 'Y', (1997, 'SEA'): 'Y', (1997, 'ATL'): 'Y', (1997, 'FLA'): 'Y', (1997, 'SFG'): 'Y',
    (1998, 'NYY'): 'Y', (1998, 'CLE'): 'Y', (1998, 'TEX'): 'Y', (1998, 'ATL'): 'Y', (1998, 'HOU'): 'Y', (1998, 'SDN'): 'Y',
    (1999, 'NYY'): 'Y', (1999, 'CLE'): 'Y', (1999, 'TEX'): 'Y', (1999, 'ATL'): 'Y', (1999, 'HOU'): 'Y', (1999, 'ARI'): 'Y',
    (2000, 'NYY'): 'Y', (2000, 'CHW'): 'Y', (2000, 'OAK'): 'Y', (2000, 'ATL'): 'Y', (2000, 'STL'): 'Y', (2000, 'SFN'): 'Y',
    (2001, 'NYY'): 'Y', (2001, 'CLE'): 'Y', (2001, 'SEA'): 'Y', (2001, 'ATL'): 'Y', (2001, 'HOU'): 'Y', (2001, 'ARI'): 'Y',
    (2002, 'NYY'): 'Y', (2002, 'MIN'): 'Y', (2002, 'OAK'): 'Y', (2002, 'ATL'): 'Y', (2002, 'STL'): 'Y', (2002, 'ARI'): 'Y',
    (2003, 'NYY'): 'Y', (2003, 'MIN'): 'Y', (2003, 'OAK'): 'Y', (2003, 'ATL'): 'Y', (2003, 'CHC'): 'Y', (2003, 'SFN'): 'Y',
    (2004, 'NYY'): 'Y', (2004, 'MIN'): 'Y', (2004, 'ANA'): 'Y', (2004, 'ATL'): 'Y', (2004, 'STL'): 'Y', (2004, 'LAD'): 'Y',
    (2005, 'NYY'): 'Y', (2005, 'CHW'): 'Y', (2005, 'LAA'): 'Y', (2005, 'ATL'): 'Y', (2005, 'STL'): 'Y', (2005, 'SDN'): 'Y',
    (2006, 'NYY'): 'Y', (2006, 'MIN'): 'Y', (2006, 'OAK'): 'Y', (2006, 'NYM'): 'Y', (2006, 'STL'): 'Y', (2006, 'SDN'): 'Y',
    (2007, 'BOS'): 'Y', (2007, 'CLE'): 'Y', (2007, 'LAA'): 'Y', (2007, 'PHI'): 'Y', (2007, 'CHC'): 'Y', (2007, 'ARI'): 'Y',
    (2008, 'TBR'): 'Y', (2008, 'CHW'): 'Y', (2008, 'LAA'): 'Y', (2008, 'PHI'): 'Y', (2008, 'CHC'): 'Y', (2008, 'LAD'): 'Y',
    (2009, 'NYY'): 'Y', (2009, 'MIN'): 'Y', (2009, 'LAA'): 'Y', (2009, 'PHI'): 'Y', (2009, 'STL'): 'Y', (2009, 'LAD'): 'Y',
    (2010, 'MIN'): 'Y', (2010, 'TEX'): 'Y', (2010, 'TBR'): 'Y', (2010, 'PHI'): 'Y', (2010, 'CIN'): 'Y', (2010, 'SFN'): 'Y',
    (2011, 'NYY'): 'Y', (2011, 'DET'): 'Y', (2011, 'TEX'): 'Y', (2011, 'PHI'): 'Y', (2011, 'MIL'): 'Y', (2011, 'ARI'): 'Y',
    (2012, 'NYY'): 'Y', (2012, 'DET'): 'Y', (2012, 'OAK'): 'Y', (2012, 'WSN'): 'Y', (2012, 'CIN'): 'Y', (2012, 'SFN'): 'Y',
    (2013, 'BOS'): 'Y', (2013, 'DET'): 'Y', (2013, 'OAK'): 'Y', (2013, 'ATL'): 'Y', (2013, 'STL'): 'Y', (2013, 'LAD'): 'Y',
    (2014, 'BAL'): 'Y', (2014, 'DET'): 'Y', (2014, 'LAA'): 'Y', (2014, 'WSN'): 'Y', (2014, 'STL'): 'Y', (2014, 'LAD'): 'Y',
    (2015, 'TOR'): 'Y', (2015, 'KCR'): 'Y', (2015, 'TEX'): 'Y', (2015, 'NYM'): 'Y', (2015, 'STL'): 'Y', (2015, 'LAD'): 'Y',
    (2016, 'BOS'): 'Y', (2016, 'CLE'): 'Y', (2016, 'TEX'): 'Y', (2016, 'WSN'): 'Y', (2016, 'CHC'): 'Y', (2016, 'LAD'): 'Y',
    (2017, 'BOS'): 'Y', (2017, 'CLE'): 'Y', (2017, 'HOU'): 'Y', (2017, 'WSN'): 'Y', (2017, 'CHC'): 'Y', (2017, 'LAD'): 'Y',
    (2018, 'BOS'): 'Y', (2018, 'CLE'): 'Y', (2018, 'HOU'): 'Y', (2018, 'ATL'): 'Y', (2018, 'MIL'): 'Y', (2018, 'LAD'): 'Y',
    (2019, 'NYY'): 'Y', (2019, 'MIN'): 'Y', (2019, 'HOU'): 'Y', (2019, 'ATL'): 'Y', (2019, 'STL'): 'Y', (2019, 'LAD'): 'Y',
    (2020, 'TBR'): 'Y', (2020, 'MIN'): 'Y', (2020, 'OAK'): 'Y', (2020, 'ATL'): 'Y', (2020, 'CHC'): 'Y', (2020, 'LAD'): 'Y',
    (2021, 'TBR'): 'Y', (2021, 'CHW'): 'Y', (2021, 'HOU'): 'Y', (2021, 'ATL'): 'Y', (2021, 'MIL'): 'Y', (2021, 'SFG'): 'Y',
    (2022, 'NYY'): 'Y', (2022, 'CLE'): 'Y', (2022, 'HOU'): 'Y', (2022, 'ATL'): 'Y', (2022, 'STL'): 'Y', (2022, 'LAD'): 'Y',
    (2023, 'BAL'): 'Y', (2023, 'MIN'): 'Y', (2023, 'HOU'): 'Y', (2023, 'ATL'): 'Y', (2023, 'MIL'): 'Y', (2023, 'LAD'): 'Y',
    (2024, 'BAL'): 'Y', (2024, 'CLE'): 'Y', (2024, 'HOU'): 'Y', (2024, 'ATL'): 'Y', (2024, 'MIL'): 'Y', (2024, 'LAD'): 'Y',
}

league_winners = {
    (1990, 'CIN'): 'Y', (1990, 'OAK'): 'Y',
    (1991, 'MIN'): 'Y', (1991, 'ATL'): 'Y',
    (1992, 'TOR'): 'Y', (1992, 'ATL'): 'Y',
    (1993, 'TOR'): 'Y', (1993, 'PHI'): 'Y',
    (1995, 'CLE'): 'Y', (1995, 'ATL'): 'Y',
    (1996, 'NYY'): 'Y', (1996, 'ATL'): 'Y',
    (1997, 'CLE'): 'Y', (1997, 'FLA'): 'Y',
    (1998, 'NYY'): 'Y', (1998, 'SDN'): 'Y',
    (1999, 'NYY'): 'Y', (1999, 'ATL'): 'Y',
    (2000, 'NYY'): 'Y', (2000, 'NYN'): 'Y',
    (2001, 'NYY'): 'Y', (2001, 'ARI'): 'Y',
    (2002, 'ANA'): 'Y', (2002, 'SFN'): 'Y',
    (2003, 'NYY'): 'Y', (2003, 'FLA'): 'Y',
    (2004, 'BOS'): 'Y', (2004, 'STL'): 'Y',
    (2005, 'CHW'): 'Y', (2005, 'HOU'): 'Y',
    (2006, 'DET'): 'Y', (2006, 'STL'): 'Y',
    (2007, 'BOS'): 'Y', (2007, 'COL'): 'Y',
    (2008, 'TBR'): 'Y', (2008, 'PHI'): 'Y',
    (2009, 'NYY'): 'Y', (2009, 'PHI'): 'Y',
    (2010, 'TEX'): 'Y', (2010, 'SFN'): 'Y',
    (2011, 'TEX'): 'Y', (2011, 'STL'): 'Y',
    (2012, 'DET'): 'Y', (2012, 'SFN'): 'Y',
    (2013, 'BOS'): 'Y', (2013, 'STL'): 'Y',
    (2014, 'KCR'): 'Y', (2014, 'SFN'): 'Y',
    (2015, 'KCR'): 'Y', (2015, 'NYN'): 'Y',
    (2016, 'CLE'): 'Y', (2016, 'CHC'): 'Y',
    (2017, 'HOU'): 'Y', (2017, 'LAD'): 'Y',
    (2018, 'BOS'): 'Y', (2018, 'LAD'): 'Y',
    (2019, 'HOU'): 'Y', (2019, 'WSN'): 'Y',
    (2020, 'TBR'): 'Y', (2020, 'LAD'): 'Y',
    (2021, 'HOU'): 'Y', (2021, 'ATL'): 'Y',
    (2022, 'HOU'): 'Y', (2022, 'PHI'): 'Y',
    (2023, 'TEX'): 'Y', (2023, 'ARI'): 'Y',
    (2024, 'BAL'): 'Y', (2024, 'LAD'): 'Y',
}
world_series_winners = {
    (1990, 'CIN'): 'Y',
    (1991, 'MIN'): 'Y',
    (1992, 'TOR'): 'Y',
    (1993, 'TOR'): 'Y',
    (1995, 'ATL'): 'Y',
    (1996, 'NYY'): 'Y',
    (1997, 'FLA'): 'Y',
    (1998, 'NYY'): 'Y',
    (1999, 'NYY'): 'Y',
    (2000, 'NYY'): 'Y',
    (2001, 'ARI'): 'Y',
    (2002, 'ANA'): 'Y',
    (2003, 'FLA'): 'Y',
    (2004, 'BOS'): 'Y',
    (2005, 'CHW'): 'Y',
    (2006, 'STL'): 'Y',
    (2007, 'BOS'): 'Y',
    (2008, 'PHI'): 'Y',
    (2009, 'NYY'): 'Y',
    (2010, 'SFN'): 'Y',
    (2011, 'STL'): 'Y',
    (2012, 'SFN'): 'Y',
    (2013, 'BOS'): 'Y',
    (2014, 'SFN'): 'Y',
    (2015, 'KCR'): 'Y',
    (2016, 'CHC'): 'Y',
    (2017, 'HOU'): 'Y',
    (2018, 'BOS'): 'Y',
    (2019, 'WSN'): 'Y',
    (2020, 'LAD'): 'Y',
    (2021, 'ATL'): 'Y',
    (2022, 'HOU'): 'Y',
    (2023, 'TEX'): 'Y',
    (2024, 'LAD'): 'Y',
}

final_df['DivisionWinner'] = final_df.apply(lambda row: division_winners.get((row['Year'], row['TeamID']), 'N'), axis=1)
final_df['LeagueWinner'] = final_df.apply(lambda row: league_winners.get((row['Year'], row['TeamID']), 'N'), axis=1)
final_df['WorldSeriesWinner'] = final_df.apply(lambda row: world_series_winners.get((row['Year'], row['TeamID']), 'N'), axis=1)

In [38]:
final_fix_map = {
    'CHN': 'CHC',
    'CHA': 'CHW',
    'NYN': 'NYM',
    'NYA': 'NYY',
    'LAN': 'LAD',
    'SDN': 'SDP',
    'SFN': 'SFG',
    'SLN': 'STL',
    'ML4': 'MIL',
    'KCA': 'KCR',
    'CAL': 'LAA',   # California Angels --> LAA
    'ANA': 'LAA',   # Anaheim Angels --> LAA
    'TBA': 'TBR',   # Tampa Bay Devil Rays --> Rays
    'FLO': 'MIA',   # Florida Marlins --> Miami Marlins
    'MON': 'WSN',   # Montreal Expos --> Washington Nationals
    'WAS': 'WSN',   # Sometimes Washington gets labeled WAS
}

final_df['TeamID'] = final_df['TeamID'].replace(final_fix_map)


In [40]:
final_df.to_csv("MLB_Data1.csv", encoding='utf-8',index=False)

### Scraping Team Pitching stats

### Test for 2024

In [44]:
# Target year
year = 2024
url = f"https://www.baseball-reference.com/leagues/majors/{year}-standard-pitching.shtml"

# Read all tables on the page
tables = pd.read_html(url)

# Look for the table with the 'Tm' column
pitching_df = None
for table in tables:
    if 'Tm' in table.columns:
        pitching_df = table
        break

# Check if table was found
if pitching_df is None:
    raise ValueError("Pitching table not found on the page.")

# Drop League Total row
pitching_df = pitching_df[pitching_df['Tm'] != 'League Total']

# Add year column
pitching_df['Year'] = year

# Rename and keep only relevant columns
pitching_df = pitching_df.rename(columns={
    'Tm': 'TeamName',
    'ERA': 'ERA',
    'SO': 'StrikeoutsPitching',
    'BB': 'WalksPitching',
    'HR': 'HRAllowed'
})

# Select final columns
pitching_df = pitching_df[['Year', 'TeamName', 'ERA', 'StrikeoutsPitching', 'WalksPitching', 'HRAllowed']]

# Preview
print("2024 Team Pitching Stats:")
print(pitching_df.head())



2024 Team Pitching Stats:
   Year              TeamName   ERA StrikeoutsPitching WalksPitching HRAllowed
0  2024  Arizona Diamondbacks  4.62               1313           481       181
1  2024        Atlanta Braves  3.49               1553           449       150
2  2024     Baltimore Orioles  3.94               1380           481       175
3  2024        Boston Red Sox  4.04               1353           461       189
4  2024          Chicago Cubs  3.78               1348           485       181


### Looping through 1990-2024

In [47]:
pitching_data = []

for year in range(1990, 2025):  # Loop through 1990 to 2024
    try:
        url = f"https://www.baseball-reference.com/leagues/majors/{year}-standard-pitching.shtml"
        tables = pd.read_html(url)

        # Find the pitching table
        df = None
        for table in tables:
            if 'Tm' in table.columns:
                df = table
                break

        if df is None:
            print(f"No table found for {year} — skipped.")
            continue

        # Drop League Total row
        df = df[df['Tm'] != 'League Total']
        df['Year'] = year

        # Rename columns
        df = df.rename(columns={
            'Tm': 'TeamName',
            'ERA': 'ERA',
            'SO': 'StrikeoutsPitching',
            'BB': 'WalksPitching',
            'HR': 'HRAllowed'
        })

        # Keep only relevant columns
        df = df[['Year', 'TeamName', 'ERA', 'StrikeoutsPitching', 'WalksPitching', 'HRAllowed']]
        pitching_data.append(df)

        print(f"Scraped {year}")
        time.sleep(random.uniform(1.5, 3))  # Be respectful to server

    except Exception as e:
        print(f"Failed to scrape {year}: {e}")

# Combine all years into one DataFrame
pitching_df = pd.concat(pitching_data, ignore_index=True)

# Show a preview of the full result
print("\n Preview of team pitching stats from 1990 to 2024:")
print(pitching_df.head(10))
print(len(pitching_df))


Scraped 1990
Scraped 1991
Scraped 1992
Scraped 1993
Scraped 1994
Scraped 1995
Scraped 1996
Scraped 1997
Scraped 1998
Scraped 1999
Scraped 2000
Scraped 2001
Scraped 2002
Scraped 2003
Scraped 2004
Scraped 2005
Scraped 2006
Scraped 2007
Scraped 2008
Scraped 2009
Scraped 2010
Scraped 2011
Scraped 2012
Scraped 2013
Scraped 2014
Scraped 2015
Scraped 2016
Scraped 2017
Scraped 2018
Scraped 2019
Scraped 2020
Scraped 2021
Scraped 2022
Scraped 2023
Scraped 2024

 Preview of team pitching stats from 1990 to 2024:
   Year           TeamName   ERA StrikeoutsPitching WalksPitching HRAllowed
0  1990     Atlanta Braves  4.58                938           579       128
1  1990  Baltimore Orioles  4.04                776           537       161
2  1990     Boston Red Sox  3.72                997           519        92
3  1990  California Angels  3.79                944           544       106
4  1990       Chicago Cubs  4.34                877           572       121
5  1990  Chicago White Sox  3.61     

### Add TeamID to scrapped data

In [50]:
team_name_to_id = {
    'Arizona Diamondbacks': 'ARI',
    'Atlanta Braves': 'ATL',
    'Baltimore Orioles': 'BAL',
    'Boston Red Sox': 'BOS',
    'California Angels': 'LAA',   # old name, now LAA
    'Chicago Cubs': 'CHC',
    'Chicago White Sox': 'CHW',
    'Cincinnati Reds': 'CIN',
    'Cleveland Guardians': 'CLE',
    'Cleveland Indians': 'CLE',
    'Colorado Rockies': 'COL',
    'Detroit Tigers': 'DET',
    'Florida Marlins': 'MIA',      # old name, now MIA
    'Miami Marlins': 'MIA',
    'Houston Astros': 'HOU',
    'Kansas City Royals': 'KCR',
    'Los Angeles Angels': 'LAA',
    'Los Angeles Angels of Anaheim': 'LAA',
    'Anaheim Angels': 'LAA',
    'Los Angeles Dodgers': 'LAD',
    'Milwaukee Brewers': 'MIL',
    'Minnesota Twins': 'MIN',
    'Montreal Expos': 'WSN',        # old name, now WSN
    'New York Mets': 'NYM',
    'New York Yankees': 'NYY',
    'Oakland Athletics': 'OAK',
    'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT',
    'San Diego Padres': 'SDP',
    'San Francisco Giants': 'SFG',
    'Seattle Mariners': 'SEA',
    'St. Louis Cardinals': 'STL',
    'Tampa Bay Devil Rays': 'TBR',  # old name, now TBR
    'Tampa Bay Rays': 'TBR',
    'Texas Rangers': 'TEX',
    'Toronto Blue Jays': 'TOR',
    'Washington Nationals': 'WSN',
}
pitching_df['TeamID'] = pitching_df['TeamName'].map(team_name_to_id)


### Merging the data

In [52]:
final_df = final_df.merge(
    pitching_df[['Year', 'TeamID', 'ERA', 'StrikeoutsPitching', 'WalksPitching', 'HRAllowed']],
    on=['Year', 'TeamID'],
    how='left'   # <- VERY important: keeps all teams even if pitching is missing
)

In [54]:
missing_rows = final_df[final_df['ERA'].isnull()]

print(f"Rows with missing ERA after merge: {missing_rows.shape[0]}")  # How many missing?
print(missing_rows[['Year', 'TeamID']])                                # Show missing ones


Rows with missing ERA after merge: 0
Empty DataFrame
Columns: [Year, TeamID]
Index: []


In [56]:
final_df['HomeRuns'] = pd.to_numeric(final_df['HomeRuns'], errors='coerce')
final_df['HRAllowed'] = pd.to_numeric(final_df['HRAllowed'], errors='coerce')

final_df['HR_diff'] = final_df['HomeRuns'] - final_df['HRAllowed']
final_df

,Year,TeamID,Conference,Division,BallPark,DivisionRank,Wins,Losses,Games,HomeRuns,...,AtBats,DivisionWinner,WildCardWinner,LeagueWinner,WorldSeriesWinner,ERA,StrikeoutsPitching,WalksPitching,HRAllowed,HR_diff
0,1990,ATL,NL,E,Truist Park,6.0,65.0,97.0,162,162,...,5504,N,NaN,N,N,4.58,938,579,128,34
1,1990,BAL,AL,E,Oriole Park at Camden Yards,5.0,76.0,85.0,161,132,...,5410,N,NaN,N,N,4.04,776,537,161,-29
2,1990,BOS,AL,E,Fenway Park,1.0,88.0,74.0,162,106,...,5516,Y,NaN,N,N,3.72,997,519,92,14
3,1990,LAA,AL,W,Anaheim Stadium,4.0,80.0,82.0,162,147,...,5570,N,NaN,N,N,3.79,944,544,106,41
4,1990,CHW,AL,C,Comiskey Park / Guaranteed Rate Field,2.0,94.0,68.0,162,106,...,5402,N,NaN,N,N,3.61,914,548,106,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,2024,STL,NL,C,Busch Stadium,NaN,NaN,NaN,162,165,...,5507,N,NaN,N,N,4.04,1308,454,174,-9
1024,2024,TBR,AL,E,Tropicana Field,NaN,NaN,NaN,162,147,...,5389,N,NaN,N,N,3.77,1406,445,196,-49
1025,2024,TEX,AL,W,Globe Life Field,NaN,NaN,NaN,162,176,...,5472,N,NaN,N,N,4.35,1371,530,198,-22
1026,2024,TOR,AL,E,Rogers Centre,NaN,NaN,NaN,162,156,...,5410,N,NaN,N,N,4.29,1314,503,208,-52


In [58]:
import pandas as pd

years = list(range(2016, 2025))
all_standings = []

for year in years:
    url = f"https://www.baseball-reference.com/leagues/majors/{year}-standings.shtml"
    tables = pd.read_html(url)
    
    # Keep only tables that have W and L columns
    for table in tables:
        if 'W' in table.columns and 'L' in table.columns and 'Tm' in table.columns:
            small_table = table[['Tm', 'W', 'L']].copy()
            small_table['Year'] = year
            all_standings.append(small_table)

standings_df = pd.concat(all_standings, ignore_index=True)

# Clean team names (remove asterisks etc.)
standings_df['Tm'] = standings_df['Tm'].str.replace(r'\*|\(\d+\)', '', regex=True).str.strip()

print(standings_df)


                       Tm   W    L  Year
0          Boston Red Sox  93   69  2016
1       Baltimore Orioles  89   73  2016
2       Toronto Blue Jays  89   73  2016
3        New York Yankees  84   78  2016
4          Tampa Bay Rays  68   94  2016
..                    ...  ..  ...   ...
265   Los Angeles Dodgers  98   64  2024
266      San Diego Padres  93   69  2024
267  Arizona Diamondbacks  89   73  2024
268  San Francisco Giants  80   82  2024
269      Colorado Rockies  61  101  2024

[270 rows x 4 columns]


In [60]:
# Mapping team names to TeamID abbreviations
team_name_to_id = {
    'Arizona Diamondbacks': 'ARI',
    'Atlanta Braves': 'ATL',
    'Baltimore Orioles': 'BAL',
    'Boston Red Sox': 'BOS',
    'Chicago Cubs': 'CHC',
    'Chicago White Sox': 'CHW',
    'Cincinnati Reds': 'CIN',
    'Cleveland Guardians': 'CLE',  # 2022-present
    'Cleveland Indians': 'CLE',     # 2016-2021
    'Colorado Rockies': 'COL',
    'Detroit Tigers': 'DET',
    'Houston Astros': 'HOU',
    'Kansas City Royals': 'KCR',
    'Los Angeles Angels': 'LAA',
    'Los Angeles Dodgers': 'LAD',
    'Miami Marlins': 'MIA',
    'Milwaukee Brewers': 'MIL',
    'Minnesota Twins': 'MIN',
    'New York Yankees': 'NYY',
    'New York Mets': 'NYM',
    'Oakland Athletics': 'OAK',
    'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT',
    'San Diego Padres': 'SDP',
    'San Francisco Giants': 'SFG',
    'Seattle Mariners': 'SEA',
    'St. Louis Cardinals': 'STL',
    'Tampa Bay Rays': 'TBR',
    'Texas Rangers': 'TEX',
    'Toronto Blue Jays': 'TOR',
    'Washington Nationals': 'WSN'
}

# Map team names to TeamID
standings_df['TeamID'] = standings_df['Tm'].map(team_name_to_id)

print(standings_df)


                       Tm   W    L  Year TeamID
0          Boston Red Sox  93   69  2016    BOS
1       Baltimore Orioles  89   73  2016    BAL
2       Toronto Blue Jays  89   73  2016    TOR
3        New York Yankees  84   78  2016    NYY
4          Tampa Bay Rays  68   94  2016    TBR
..                    ...  ..  ...   ...    ...
265   Los Angeles Dodgers  98   64  2024    LAD
266      San Diego Padres  93   69  2024    SDP
267  Arizona Diamondbacks  89   73  2024    ARI
268  San Francisco Giants  80   82  2024    SFG
269      Colorado Rockies  61  101  2024    COL

[270 rows x 5 columns]


In [62]:
standings_df.rename(columns={'W': 'Wins', 'L': 'Losses',}, inplace=True)
standings_df = standings_df.drop(columns=['Tm'])

standings_df

,Wins,Losses,Year,TeamID
0,93,69,2016,BOS
1,89,73,2016,BAL
2,89,73,2016,TOR
3,84,78,2016,NYY
4,68,94,2016,TBR
...,...,...,...,...
265,98,64,2024,LAD
266,93,69,2024,SDP
267,89,73,2024,ARI
268,80,82,2024,SFG


In [65]:
print("Final DF Columns:", final_df.columns.tolist())
print("Standings DF Columns:", standings_df.columns.tolist())
final_df

Final DF Columns: ['Year', 'TeamID', 'Conference', 'Division', 'BallPark', 'DivisionRank', 'Wins', 'Losses', 'Games', 'HomeRuns', 'Walks', 'Strikeouts', 'Runs', 'Hits', 'AtBats', 'DivisionWinner', 'WildCardWinner', 'LeagueWinner', 'WorldSeriesWinner', 'ERA', 'StrikeoutsPitching', 'WalksPitching', 'HRAllowed', 'HR_diff']
Standings DF Columns: ['Wins', 'Losses', 'Year', 'TeamID']


,Year,TeamID,Conference,Division,BallPark,DivisionRank,Wins,Losses,Games,HomeRuns,...,AtBats,DivisionWinner,WildCardWinner,LeagueWinner,WorldSeriesWinner,ERA,StrikeoutsPitching,WalksPitching,HRAllowed,HR_diff
0,1990,ATL,NL,E,Truist Park,6.0,65.0,97.0,162,162,...,5504,N,NaN,N,N,4.58,938,579,128,34
1,1990,BAL,AL,E,Oriole Park at Camden Yards,5.0,76.0,85.0,161,132,...,5410,N,NaN,N,N,4.04,776,537,161,-29
2,1990,BOS,AL,E,Fenway Park,1.0,88.0,74.0,162,106,...,5516,Y,NaN,N,N,3.72,997,519,92,14
3,1990,LAA,AL,W,Anaheim Stadium,4.0,80.0,82.0,162,147,...,5570,N,NaN,N,N,3.79,944,544,106,41
4,1990,CHW,AL,C,Comiskey Park / Guaranteed Rate Field,2.0,94.0,68.0,162,106,...,5402,N,NaN,N,N,3.61,914,548,106,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,2024,STL,NL,C,Busch Stadium,NaN,NaN,NaN,162,165,...,5507,N,NaN,N,N,4.04,1308,454,174,-9
1024,2024,TBR,AL,E,Tropicana Field,NaN,NaN,NaN,162,147,...,5389,N,NaN,N,N,3.77,1406,445,196,-49
1025,2024,TEX,AL,W,Globe Life Field,NaN,NaN,NaN,162,176,...,5472,N,NaN,N,N,4.35,1371,530,198,-22
1026,2024,TOR,AL,E,Rogers Centre,NaN,NaN,NaN,162,156,...,5410,N,NaN,N,N,4.29,1314,503,208,-52


In [67]:
# 1. Set indexes to match
standings_update = standings_df.set_index(['Year', 'TeamID'])

# 2. Update Wins and Losses ONLY
final_df.set_index(['Year', 'TeamID'], inplace=True)
final_df.update(standings_update[['Wins', 'Losses']])
final_df.reset_index(inplace=True)
final_df

,Year,TeamID,Conference,Division,BallPark,DivisionRank,Wins,Losses,Games,HomeRuns,...,AtBats,DivisionWinner,WildCardWinner,LeagueWinner,WorldSeriesWinner,ERA,StrikeoutsPitching,WalksPitching,HRAllowed,HR_diff
0,1990,ATL,NL,E,Truist Park,6.0,65.0,97.0,162,162,...,5504,N,NaN,N,N,4.58,938,579,128,34
1,1990,BAL,AL,E,Oriole Park at Camden Yards,5.0,76.0,85.0,161,132,...,5410,N,NaN,N,N,4.04,776,537,161,-29
2,1990,BOS,AL,E,Fenway Park,1.0,88.0,74.0,162,106,...,5516,Y,NaN,N,N,3.72,997,519,92,14
3,1990,LAA,AL,W,Anaheim Stadium,4.0,80.0,82.0,162,147,...,5570,N,NaN,N,N,3.79,944,544,106,41
4,1990,CHW,AL,C,Comiskey Park / Guaranteed Rate Field,2.0,94.0,68.0,162,106,...,5402,N,NaN,N,N,3.61,914,548,106,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,2024,STL,NL,C,Busch Stadium,NaN,83.0,79.0,162,165,...,5507,N,NaN,N,N,4.04,1308,454,174,-9
1024,2024,TBR,AL,E,Tropicana Field,NaN,80.0,82.0,162,147,...,5389,N,NaN,N,N,3.77,1406,445,196,-49
1025,2024,TEX,AL,W,Globe Life Field,NaN,78.0,84.0,162,176,...,5472,N,NaN,N,N,4.35,1371,530,198,-22
1026,2024,TOR,AL,E,Rogers Centre,NaN,74.0,88.0,162,156,...,5410,N,NaN,N,N,4.29,1314,503,208,-52


In [69]:
final_df = final_df.drop(columns=['DivisionRank','WildCardWinner'], errors='ignore')


In [71]:
playoff_teams = {
    1990: ['BOS', 'OAK', 'CIN', 'PIT'],
    1991: ['MIN', 'TOR', 'ATL', 'PIT'],
    1992: ['TOR', 'OAK', 'ATL', 'PIT'],
    1993: ['TOR', 'CHW', 'ATL', 'PHI'],
    1995: ['CLE', 'SEA', 'BOS', 'NYY', 'ATL', 'CIN', 'COL', 'LAD'],
    1996: ['NYY', 'TEX', 'BAL', 'CLE', 'ATL', 'STL', 'LAD', 'SDP'],
    1997: ['BAL', 'CLE', 'SEA', 'NYY', 'FLA', 'ATL', 'SFG', 'HOU'],
    1998: ['NYY', 'TEX', 'CLE', 'BOS', 'ATL', 'HOU', 'SDP', 'CHC'],
    1999: ['NYY', 'CLE', 'TEX', 'BOS', 'ATL', 'HOU', 'ARI', 'NYM'],
    2000: ['NYY', 'CLE', 'SEA', 'OAK', 'ATL', 'STL', 'SFG', 'NYM'],
    2001: ['NYY', 'OAK', 'SEA', 'CLE', 'ATL', 'STL', 'HOU', 'ARI'],
    2002: ['NYY', 'MIN', 'OAK', 'ANA', 'ATL', 'STL', 'SFG', 'ARI'],
    2003: ['NYY', 'MIN', 'BOS', 'OAK', 'ATL', 'CHC', 'FLA', 'SFG'],
    2004: ['NYY', 'MIN', 'BOS', 'ANA', 'ATL', 'LAD', 'HOU', 'STL'],
    2005: ['CHW', 'BOS', 'LAA', 'NYY', 'STL', 'HOU', 'ATL', 'SDP'],
    2006: ['OAK', 'MIN', 'DET', 'NYY', 'STL', 'SDP', 'NYM', 'LAD'],
    2007: ['BOS', 'CLE', 'LAA', 'NYY', 'ARI', 'COL', 'PHI', 'CHC'],
    2008: ['BOS', 'TBR', 'CHW', 'LAA', 'PHI', 'MIL', 'LAD', 'CHC'],
    2009: ['NYY', 'MIN', 'LAA', 'BOS', 'PHI', 'STL', 'LAD', 'COL'],
    2010: ['TEX', 'MIN', 'NYY', 'TBR', 'SFG', 'PHI', 'CIN', 'ATL'],
    2011: ['TEX', 'DET', 'NYY', 'TBR', 'STL', 'MIL', 'ARI', 'PHI'],
    2012: ['NYY', 'OAK', 'DET', 'BAL', 'TEX', 'SFG', 'CIN', 'STL', 'ATL', 'WSN'],
    2013: ['BOS', 'DET', 'OAK', 'TBR', 'CLE', 'STL', 'ATL', 'LAD', 'PIT', 'CIN'],
    2014: ['BAL', 'DET', 'LAA', 'KCR', 'OAK', 'SFG', 'LAD', 'STL', 'PIT', 'WSN'],
    2015: ['TOR', 'KCR', 'TEX', 'NYY', 'HOU', 'NYM', 'CHC', 'STL', 'PIT', 'LAD'],
    2016: ['CLE', 'BOS', 'TEX', 'TOR', 'BAL', 'CHC', 'LAD', 'WSN', 'SFG', 'NYM'],
    2017: ['HOU', 'CLE', 'BOS', 'NYY', 'MIN', 'LAD', 'WSN', 'CHC', 'ARI', 'COL'],
    2018: ['BOS', 'HOU', 'CLE', 'NYY', 'OAK', 'MIL', 'LAD', 'ATL', 'COL', 'CHC'],
    2019: ['HOU', 'NYY', 'MIN', 'TBR', 'OAK', 'LAD', 'ATL', 'STL', 'WSN', 'MIL'],
    2020: ['TBR', 'NYY', 'TOR', 'CLE', 'MIN', 'HOU', 'OAK', 'CHW', 'LAD', 'SDP', 'CHC', 'MIA', 'ATL', 'CIN', 'STL', 'MIL'],
    2021: ['HOU', 'BOS', 'CWS', 'TBR', 'NYY', 'ATL', 'LAD', 'MIL', 'SFG', 'STL'],
    2022: ['HOU', 'NYY', 'CLE', 'SEA', 'TOR', 'TBR', 'LAD', 'ATL', 'STL', 'NYM', 'PHI', 'SDP'],
    2023: ['BAL', 'HOU', 'MIN', 'TEX', 'TOR', 'TBR', 'ATL', 'LAD', 'MIL', 'PHI', 'MIA', 'ARI'],
    2024: ['LAD', 'ATL', 'MIL', 'PHI', 'SDP', 'NYM', 'BAL', 'HOU', 'DET', 'TEX', 'NYY', 'CLE'],
}


In [73]:
final_df['MadePlayoffs'] = final_df.apply(
    lambda row: 'Y' if row['TeamID'] in playoff_teams.get(row['Year'], []) else 'N',
    axis=1
)
final_df

,Year,TeamID,Conference,Division,BallPark,Wins,Losses,Games,HomeRuns,Walks,...,AtBats,DivisionWinner,LeagueWinner,WorldSeriesWinner,ERA,StrikeoutsPitching,WalksPitching,HRAllowed,HR_diff,MadePlayoffs
0,1990,ATL,NL,E,Truist Park,65.0,97.0,162,162,473,...,5504,N,N,N,4.58,938,579,128,34,N
1,1990,BAL,AL,E,Oriole Park at Camden Yards,76.0,85.0,161,132,660,...,5410,N,N,N,4.04,776,537,161,-29,N
2,1990,BOS,AL,E,Fenway Park,88.0,74.0,162,106,598,...,5516,Y,N,N,3.72,997,519,92,14,Y
3,1990,LAA,AL,W,Anaheim Stadium,80.0,82.0,162,147,566,...,5570,N,N,N,3.79,944,544,106,41,N
4,1990,CHW,AL,C,Comiskey Park / Guaranteed Rate Field,94.0,68.0,162,106,478,...,5402,N,N,N,3.61,914,548,106,0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,2024,STL,NL,C,Busch Stadium,83.0,79.0,162,165,478,...,5507,N,N,N,4.04,1308,454,174,-9,N
1024,2024,TBR,AL,E,Tropicana Field,80.0,82.0,162,147,493,...,5389,N,N,N,3.77,1406,445,196,-49,N
1025,2024,TEX,AL,W,Globe Life Field,78.0,84.0,162,176,490,...,5472,N,N,N,4.35,1371,530,198,-22,Y
1026,2024,TOR,AL,E,Rogers Centre,74.0,88.0,162,156,510,...,5410,N,N,N,4.29,1314,503,208,-52,N


### Saving the final merged data set

In [76]:
# Save the final merged dataset
final_df.to_csv("merged_1990_2024_data.csv", index=False)
print("Saved as merged_1990_2024_data.csv")


Saved as merged_1990_2024_data.csv
